In [1]:
!pip install --upgrade kaggle --force-reinstall


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.4/164.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.5/153.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.2/323.2 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.

In [1]:
from google.colab import files
files.upload()  # kaggle.json 파일 선택


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"icdi050227","key":"6738515dc0795c218ea602dfa0ddc433"}'}

In [2]:
!mkdir -p ~/.kaggle
!mkdir -p /root/.config/kaggle
!cp kaggle.json ~/.kaggle/
!cp kaggle.json /root/.config/kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!chmod 600 /root/.config/kaggle/kaggle.json


In [3]:
!kaggle competitions download -c state-farm-distracted-driver-detection


100% 3.99G/4.00G [00:42<00:00, 147MB/s]
100% 4.00G/4.00G [00:42<00:00, 101MB/s]


In [4]:
!unzip -q /content/state-farm-distracted-driver-detection.zip -d /content/statefarm_data


In [21]:
#기본 라이브러리 임포트
import os, random, torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from tqdm import tqdm
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import KFold

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

NUM_CLASSES = 10
BATCH_SIZE = 32
EPOCHS = 5
N_FOLDS = 5
BASE_SEED = 42  # 팀 공통 seed 기준

train_dir = "/content/statefarm_data/imgs/train"
test_dir  = "/content/statefarm_data/imgs/test"
driver_csv = "/content/statefarm_data/driver_imgs_list.csv"


Using device: cpu


In [23]:
#팀 전처리 함수 정의
def team_augmentation(w, h):
    """
    팀에서 통일된 데이터 증강 및 정규화 정책 적용
    Args:
        w, h: resize 크기
    Returns:
        transform_train: 학습용 증강 포함 transform
        transform_eval: 검증/테스트용 transform
    """
    transform_train = transforms.Compose([
        transforms.Resize((w, h)),                    # 입력 크기 통일
        transforms.RandomRotation(degrees=15),        # ±15도 회전
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.05),
        transforms.ToTensor(),                        # Tensor 변환
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])  # ImageNet 정규화
    ])
    transform_eval = transforms.Compose([
        transforms.Resize((w, h)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    return transform_train, transform_eval

# transform 불러오기 (224x224로 통일)
transform_train, transform_eval = team_augmentation(224, 224)

In [24]:
#운전자 목록 불러오기
df = pd.read_csv(driver_csv)
drivers = sorted(df['subject'].unique())  # 정렬로 순서 고정
print(f"총 운전자 수: {len(drivers)}명")

총 운전자 수: 26명


In [25]:
#커스텀 Dataset 정의
class DriverDataset(Dataset):
    """
    driver_imgs_list.csv 기반으로 이미지 로드하는 Dataset 클래스
    """
    def __init__(self, dataframe, root_dir, transform=None):
        self.dataframe = dataframe.reset_index(drop=True)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        img_path = os.path.join(self.root_dir, row['classname'], row['img'])
        img = Image.open(img_path).convert('RGB')  # 이미지 RGB 변환
        if self.transform:
            img = self.transform(img)
        label = int(row['classname'][1])  # c0~c9 → 숫자 0~9
        return img, label

In [26]:
#모델 생성 함수
def create_alexnet():
    model = models.alexnet(pretrained=True)
    for param in model.features.parameters():
        param.requires_grad = False
    model.classifier[6] = nn.Linear(4096, NUM_CLASSES)
    return model.to(device)

In [28]:
#5-fold 학습
kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=BASE_SEED)
fold_results = []
best_model_path = None
global_best_acc = 0.0

for fold, (train_idx, val_idx) in enumerate(kf.split(drivers), 1):
    seed = BASE_SEED + fold  # fold마다 고유 seed
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    # 운전자 분리
    train_drivers = np.array(drivers)[train_idx]
    val_drivers   = np.array(drivers)[val_idx]

    train_df = df[df['subject'].isin(train_drivers)]
    val_df   = df[df['subject'].isin(val_drivers)]

    print(f"\n===== Fold {fold}/{N_FOLDS} =====")
    print(f"Train drivers: {len(train_drivers)}명 | Val drivers: {len(val_drivers)}명")
    print(f"Train images: {len(train_df)} | Val images: {len(val_df)}")

    # 데이터셋 및 로더
    train_dataset = DriverDataset(train_df, train_dir, transform=transform_train)
    val_dataset   = DriverDataset(val_df,   train_dir, transform=transform_eval)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # 모델, 손실함수, 옵티마이저
    model = create_alexnet()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    best_val_acc = 0.0

    # ---- 학습 루프 ----
    for epoch in range(EPOCHS):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for imgs, labels in tqdm(train_loader, desc=f"Fold {fold} | Epoch {epoch+1}/{EPOCHS}"):
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_acc = 100 * correct / total

        # ---- 검증 ----
        model.eval()
        val_correct, val_total = 0, 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_acc = 100 * val_correct / val_total
        print(f"Fold {fold} | Epoch {epoch+1}/{EPOCHS} | Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

        # ---- Best 모델 저장 ----
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            model_path = f"alexnet_fold{fold}_best.pth"
            torch.save(model.state_dict(), model_path)

    print(f"🎯 Fold {fold} 완료! Best Validation Accuracy: {best_val_acc:.2f}%")
    fold_results.append(best_val_acc)

    # 가장 높은 val_acc 모델 기록
    if best_val_acc > global_best_acc:
        global_best_acc = best_val_acc
        best_model_path = model_path


===== Fold 1/5 =====
Train drivers: 20명 | Val drivers: 6명
Train images: 17446 | Val images: 4978


Fold 1 | Epoch 1/5: 100%|██████████| 546/546 [15:30<00:00,  1.70s/it]


Fold 1 | Epoch 1/5 | Train Acc: 73.62% | Val Acc: 67.18%


Fold 1 | Epoch 2/5: 100%|██████████| 546/546 [16:07<00:00,  1.77s/it]


Fold 1 | Epoch 2/5 | Train Acc: 87.30% | Val Acc: 60.77%


Fold 1 | Epoch 3/5:  66%|██████▌   | 360/546 [11:00<05:41,  1.83s/it]


KeyboardInterrupt: 

In [17]:
#Fold 평균 및 최고 모델 표시
avg_acc = np.mean(fold_results)
print("\n=======================================")
print(f" 5-Fold 평균 Validation Accuracy: {avg_acc:.2f}%")
print(f"최고 성능 모델: {best_model_path} ({global_best_acc:.2f}%)")
print("=======================================")


NameError: name 'avg_acc' is not defined

In [ ]:
#Test Dataset + 예측 (최고 모델 1개만 사용)
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, folder, transform=None):
        self.folder = folder
        self.imgs = sorted([f for f in os.listdir(folder) if f.endswith('.jpg')])
        self.transform = transform
    def __len__(self):
        return len(self.imgs)
    def __getitem__(self, idx):
        img_path = os.path.join(self.folder, self.imgs[idx])
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, self.imgs[idx]

test_dataset = TestDataset(test_dir, transform=transform_eval)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
print(f"✅ Test Dataset 준비 완료: {len(test_dataset)}개 이미지")

# ---- 최고 성능 모델 로드 ----
model = create_alexnet()
model.load_state_dict(torch.load(best_model_path, map_location=device))
model.eval()

# ---- Test 예측 ----
results = []
with torch.no_grad():
    for imgs, names in tqdm(test_loader, desc=f"Predicting with Best Model"):
        imgs = imgs.to(device)
        outputs = torch.softmax(model(imgs), dim=1)  # 확률값
        outputs = outputs.cpu().numpy()
        for name, probs in zip(names, outputs):
            results.append([name] + probs.tolist())

# ---- CSV 생성 ----
columns = ["img"] + [f"c{i}" for i in range(NUM_CLASSES)]
df_sub = pd.DataFrame(results, columns=columns)
df_sub.to_csv("alexnet_team_base_submission.csv", index=False)

print("\nsubmission.csv 생성 완료!")
print(f"사용된 모델: {best_model_path}")
print(df_sub.head())
